In this notebook we'll be building our first LSTM model with the meteorological and site readings from the S.H Bell monitoring site. We'll first need to join all of the data and do some basic data cleaning and preparation.

In [51]:
import numpy as np
import pandas as pd
from datetime import datetime

### Data Preparation

In [52]:
samples = pd.read_excel('data/sample_readings/shb_summary_of_all_continuous_data_november_2019_for_website_public_0.xlsx')
sample_copy = samples.copy()

In [53]:
samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24120 entries, 0 to 24119
Data columns (total 29 columns):
Transaction Type                     24120 non-null object
Action Code                          24120 non-null object
State Code                           24120 non-null int64
County Code                          24120 non-null int64
Site ID                              24120 non-null int64
Parameter                            24120 non-null int64
POC                                  24120 non-null int64
Sample Duration                      24120 non-null int64
Unit                                 24120 non-null int64
Method                               24120 non-null int64
Date                                 24120 non-null int64
Start Time                           24120 non-null object
Sample Value                         23902 non-null float64
Null Data Code                       299 non-null object
Sampling Frequency                   0 non-null float64
Monitor Protocol (M

We'll drop all of the columns that are completely empty and then look at the ones with a few non-null values to see if they're relevant.

In [54]:
samples.dropna(axis=1, how='all', inplace=True)

In [55]:
samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24120 entries, 0 to 24119
Data columns (total 15 columns):
Transaction Type    24120 non-null object
Action Code         24120 non-null object
State Code          24120 non-null int64
County Code         24120 non-null int64
Site ID             24120 non-null int64
Parameter           24120 non-null int64
POC                 24120 non-null int64
Sample Duration     24120 non-null int64
Unit                24120 non-null int64
Method              24120 non-null int64
Date                24120 non-null int64
Start Time          24120 non-null object
Sample Value        23902 non-null float64
Null Data Code      299 non-null object
Qualifier 1         50 non-null float64
dtypes: float64(2), int64(9), object(4)
memory usage: 2.8+ MB


In [56]:
samples['Qualifier 1'].value_counts()

9.0    50
Name: Qualifier 1, dtype: int64

In [57]:
samples['Null Data Code'].value_counts()

BA    188
BC     83
AN     18
AV      6
AM      2
AZ      2
Name: Null Data Code, dtype: int64

Descriptions of the qualifiers and null data codes can be found here: https://aqs.epa.gov/aqsweb/documents/codetables/qualifiers.html. According to this table, the values in the 'Qualifier 1' column were detected as negative. Since it doesn't make sense to have a negative reading of a PM concentration, they were instead reported as being 0. The readings in the 'Null Data Code' column are as follows:

- AM: Miscellaneous Void
- AN: Machine Malfunction
- AV: Power Failure
- AZ: Q C Audit
- BA: Maintenance/Routine Repairs
- BC: Multi-point Calibration

There doesn't seem to be any way to really deal with this null values, To make sure that we still have hourly data, we'll do some interpolation. First, let's make sure that all of the values with a null data code are actually NaNs. If they aren't, we'll set them to NaN and then resample. The same will be done for for the 'Qualifier 1' column.

In [58]:
samples.loc[samples['Null Data Code'].notnull(), 'Sample Value'] = np.nan
samples.loc[samples['Qualifier 1'].notnull(), 'Sample Value'] = np.nan

In [59]:
samples[['Sample Value', 'Null Data Code', 'Qualifier 1']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24120 entries, 0 to 24119
Data columns (total 3 columns):
Sample Value      23771 non-null float64
Null Data Code    299 non-null object
Qualifier 1       50 non-null float64
dtypes: float64(2), object(1)
memory usage: 565.4+ KB


In [60]:
# Checking to see if we have the correct number of null values
23771 + 50 + 299 == 24120

True

We'll now set the index to the time of measurement and drop the irrelevant columns. It turns out that some of the values in the 'Start Time' column have dates and time, while the rest only have times. In order to address this, we'll create dates for these values based on what is in the 'Date' column. Then, we'll add prepend these to the 'Start Time' column and drop the 'Date' column before setting 'Start Time' as the index. This will be much faster than converting the entire 'Date' column to the datetime type and adding the times from 'Start Time'.

In [83]:
samples['Start Time'].apply(lambda x: type(x)).value_counts()
# samples.set_index('Start Time', inplace=True)

<class 'datetime.time'>        21192
<class 'datetime.datetime'>     2928
Name: Start Time, dtype: int64

In [96]:
samples['Date'][2928:].head()

2928    20170701
2929    20170701
2930    20170701
2931    20170701
2932    20170701
Name: Date, dtype: int64

[datetime.datetime(2017, 7, 1, 0, 0),
 datetime.datetime(2017, 7, 1, 1, 0),
 datetime.datetime(2017, 7, 1, 2, 0),
 datetime.datetime(2017, 7, 1, 3, 0),
 datetime.datetime(2017, 7, 1, 4, 0),
 datetime.datetime(2017, 7, 1, 5, 0),
 datetime.datetime(2017, 7, 1, 6, 0),
 datetime.datetime(2017, 7, 1, 7, 0),
 datetime.datetime(2017, 7, 1, 8, 0),
 datetime.datetime(2017, 7, 1, 9, 0),
 datetime.datetime(2017, 7, 1, 10, 0),
 datetime.datetime(2017, 7, 1, 11, 0),
 datetime.datetime(2017, 7, 1, 12, 0),
 datetime.datetime(2017, 7, 1, 13, 0),
 datetime.datetime(2017, 7, 1, 14, 0),
 datetime.datetime(2017, 7, 1, 15, 0),
 datetime.datetime(2017, 7, 1, 16, 0),
 datetime.datetime(2017, 7, 1, 17, 0),
 datetime.datetime(2017, 7, 1, 18, 0),
 datetime.datetime(2017, 7, 1, 19, 0),
 datetime.datetime(2017, 7, 1, 20, 0),
 datetime.datetime(2017, 7, 1, 21, 0),
 datetime.datetime(2017, 7, 1, 22, 0),
 datetime.datetime(2017, 7, 1, 23, 0),
 datetime.datetime(2017, 7, 2, 0, 0),
 datetime.datetime(2017, 7, 2, 1, 0)

In [108]:
samples['Start Time'][2928:]

2928     00:00:00
2929     01:00:00
2930     02:00:00
2931     03:00:00
2932     04:00:00
2933     05:00:00
2934     06:00:00
2935     07:00:00
2936     08:00:00
2937     09:00:00
2938     10:00:00
2939     11:00:00
2940     12:00:00
2941     13:00:00
2942     14:00:00
2943     15:00:00
2944     16:00:00
2945     17:00:00
2946     18:00:00
2947     19:00:00
2948     20:00:00
2949     21:00:00
2950     22:00:00
2951     23:00:00
2952     00:00:00
2953     01:00:00
2954     02:00:00
2955     03:00:00
2956     04:00:00
2957     05:00:00
           ...   
24090    18:00:00
24091    19:00:00
24092    20:00:00
24093    21:00:00
24094    22:00:00
24095    23:00:00
24096    00:00:00
24097    01:00:00
24098    02:00:00
24099    03:00:00
24100    04:00:00
24101    05:00:00
24102    06:00:00
24103    07:00:00
24104    08:00:00
24105    09:00:00
24106    10:00:00
24107    11:00:00
24108    12:00:00
24109    13:00:00
24110    14:00:00
24111    15:00:00
24112    16:00:00
24113    17:00:00
24114    1

In [46]:
samples.index.apply(pd.Timestamp)

AttributeError: 'Index' object has no attribute 'apply'